In [1]:
import pandas as pd
import csv
import matplotlib.pyplot as plt
import numpy as np
import operator
import datetime
import sys
from datetime import datetime
from dateutil.parser import parse
import seaborn as sns
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#sys.argv[0] Names of the script (following ones are the arguments)

C:\Users\parmengol\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


# Dataset

In [2]:
years = ['2014','2015','2016','2017']
ls_dfs=[]
for year in years:
    df = pd.read_csv('180717_EF+RM - Obra Pública'+ ' '+ year +'.csv',engine='python')
    # y de una vez creamos una variable para el anio del contrato
    df['year']= float(year)
    ls_dfs.append(df)
df_chihuahua=pd.concat(ls_dfs, sort=False)

# Functions to standarize the names of the whole dataset

In [3]:
def pool_names(df, list_names):
    '''
    This function takes a list of colnames that contain names and pool all the names of the columns into a single list
    later on keeps just the unique names
    '''
    #Pool all names
    general = []
    for i in list_names:
        list1 = list(df[i])
        for j in list1:
            general.append(j)
    general = set(general)
    general = list(general)
    
    #Leave just strings within the list
    general = [x for x in general if isinstance(x, str) == True]
    
    return general 

In [66]:
def similarity_tuple(df,general,m):
    
    '''
    Takes column i and return a dictionary with the names as keys and with the
    tuples with other very likely same names as values
    general -> list with unique names
    m -> name of column to process
    '''
   
    
    d={}
    posicion={}
    for i, j in df.iterrows():
        posicion[j[m]] = i
    for i, j in df.iterrows():
        if isinstance(j[m], str) == True:
            list_to_compare = [x for x in general if x not in j[m]]
            ls = []
            for element in list_to_compare:
                score = fuzz.ratio(j[m], element)
                if score >93 and score <100:
                    tupla = (element)
                    ls.append(tupla)
            d[j[m]]= ls
    
    lista = []
    
    
    for i in d:
        if d[i] != []:
            tuple1 = [i,d[i][0]]
            for k in d:
                if i in d[k]:
                    tuple1.append(k)
            lista.append(tuple(tuple1))    

    for i in lista:
        for k in lista:
            if i != k and set(i)==set(k):
                lista.remove(k)
    
    lista1 = []
    num = 0
    for i in lista:
        print(i)
        num += 1
        for j in i:
            tuple1 = (num,j)
            lista1.append(tuple1)
    return lista1


In [67]:
def col_ids(df,general,colnames):
    '''
    general is the list with all the pooled names
    df dataframe
    i is the names of the column that is being processed
    '''
    for i in colnames:
        
        df1 = df.dropna(subset=[i])
               
        lista1 = similarity_tuple(df1,general,i)
        
        
        if lista1 != None:
        
            lista_empresa = pd.DataFrame(lista1)
    
            name = i + "_ID"

            lista_empresa.columns = [name, i]

            lista_empresa[name] = lista_empresa[name].apply(str)
            lista_empresa[name] = lista_empresa[name] + "m"


            df1 = df1.merge(lista_empresa, on = i, how = 'left')


            name1 = i + "ID"

            df1[name1] = df1[i].rank(method='dense').astype(str)

            df1[name1] = np.where(df1[name].isna() != True, df1[name], df1[name1])

            df1 = df1.drop_duplicates(subset=[i, name])

            df1 = df1[[i, name1]]

            df = df.merge(df1, on = i, how = "left")
        
        
    return df
        
        
    

In [68]:
def standarize_names(df, colnames):
    '''
    Standarize the names of a set of columns and asign a unique ID column for each names column in the dataset
    '''
    
    #Pool names
    general = pool_names(df, colnames)
    general = [x for x in general if str(x) != 'nan']
    #Generate columns with IDS 
    df1 = col_ids(df,general,colnames)
    
    return df1

In [73]:
list1 = ["Emp_gana","Nom_emp_propu_1","Nom_emp_propu_2","Nom_emp_propu_3","Nom_emp_propu_4","Nom_emp_propu_5"]

In [74]:
df1 = standarize_names(df_chihuahua, list1)
df1.to_csv('df_chihuahua_clean.csv')

('proyectos y construcciones valar, s.a. de c.v.', 'proyectos y construcciones hanliba, s.a. de c.v.')
('emilio rh, s. de r.l. m.i.', 'emilio rh, s. de r.l. de m.i.', 'emilio rh, s. de r.l. de m.i.')
('copsa sistemas constructivos obras y proyectos, s.a. de c.v.', 'copsa sistemas constructivos de obras y proyectos, s.a. de c.v.', 'copsa sistemas constructivos de obras y proyectos, s.a. de c.v.')
('constructora de desarrollos corporativos, s.a. de c.v.', 'constructora de desarrollos operativos, s.a. de c.v.', 'constructora de desarrollos operativos, s.a. de c.v.')
('proyectos, construcciones y servicios de chihuahua, s.a. de c.v.', 'proyectos construcciones y servicios de chihuahua, s.a. de c.v.')
('palermo construcciones s.a. de c.v.', 'palermo construcciones, s.a. de c.v.')
('luis carlos bueno gardea', 'luis carlos bueno gadea')
[(1, 'proyectos y construcciones valar, s.a. de c.v.'), (1, 'proyectos y construcciones hanliba, s.a. de c.v.'), (2, 'emilio rh, s. de r.l. m.i.'), (2, 'emili